# Cleaning Data in Python
This was done as a part of datacamp's curriculum. It's pretty useful to be reminded of the basic techniques.

In [2]:
# Import the course packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import missingno as msno
import fuzzywuzzy
import recordlinkage 

# Import the course datasets
ride_sharing = pd.read_csv('datasets/ride_sharing_new.csv', index_col = 'Unnamed: 0')
airlines = pd.read_csv('datasets/airlines_final.csv',  index_col = 'Unnamed: 0')
banking = pd.read_csv('datasets/banking_dirty.csv', index_col = 'Unnamed: 0')
restaurants = pd.read_csv('datasets/restaurants_L2.csv', index_col = 'Unnamed: 0')
restaurants_new = pd.read_csv('datasets/restaurants_L2_dirty.csv', index_col = 'Unnamed: 0')

# Common Data problems

## Converting from integer to category
It's the idea that we're changing a column from representing quantitative data to categorical data. This makes sense when a column represents a fixed set of values like enums, or a discrete set e.g. role column is 'User', 'Admin', 'Moderatotr', etc. Doing this can help reduce memory usage, increase performance, and makes our data a lot cleaner.


In [ ]:
# Creates a new column 'user_type_cat' by converting an existing column to a categorical type, 
ride_sharing['user_type_cat'] = ride_sharing['user_type'].astype("category")

# This just makes sure that our changes were actually applied to the data frame. Else our script throws an error
assert ride_sharing['user_type_cat'].dtype.name == 'category'

## Ensuring that we are working with numbers instead of strings

In [ ]:
'''
1. Strip duration column of 'minutes' string; essential to integer conversion as we can't really convert something like '  15  ' to an integer 
until we get rid of the spaces.
2. Convert duration_trim to integer
'''
ride_sharing['duration_trim'] = ride_sharing['duration'].str.strip("minutes")
ride_sharing['duration_time'] = ride_sharing['duration_trim'].astype(int)
assert ride_sharing['duration_time'].dtype == "int"

print(ride_sharing[['duration', 'duration_trim', 'duration_time']])
print(ride_sharing['duration_time'].mean())


## Range of your data 
Sometimes data is supposed to be in a set range of values. Like the registration date for a user can't be in the future, or how the ratings for a movie review can only be from 1 to 5.

### How to deal with out of range data:

1. How to deal with out of range data: Drop the data, but this is only recommended when there's only a few rows that have out of range data. 



In [ ]:
'''
- Ex.1: Bicycle tire sizes can be 26, 27, or 29. They're stored as 'categorical' values instead of numerical for some reason. However the company is now setting the new maximum tire size to be 27.

- Steps:
1. Convert tire_sizes to integer
2. Set all tire size values above 27, to 27.  
3. Remember the business rule is that tire sizes are categories rather than actual numbers to do calculations with. So convert the tire_sizes series 
back to categorical data. 
'''
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype'(int') 
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')
print(ride_sharing['tire_sizes'].describe())

In [ ]:
'''
- Ex. 2: The data pipeline feeding into our 'ride_sharing' data frame has been updated to register hwen a ride is registered. This is done in the 'ride_date' column, which is of type 'object', which is treated as strings in Pandas. A bugged happened with registration dates, so just for this one time, ensure that any values for 'ride_date' has a maximum datetime of 'today'.

- Steps:
1. It's normal for dates to be strings (object type) when first imported into a Pandas DataFrame. So we'll convert all date values in the series to a datetime object, so that we can do date related operations.
2. Save today's date as a datetime object into a variable
3. Set all future dates to today's date
4. Print hte maximum value from the ride_dt column.
'''
ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date
today = dt.date.today()
ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today
print(ride_sharing['ride_dt'].max())

## Unique constraints and duplicate values 

### What are duplicate values?
Of course a user can have the same first or last name as another user, so when we talk about duplicate values we're saying that 'most of the columns have duplicate values'. Like the first name, last name, the address, and even the weight of the users are the same. The only thing different is height, but still that's pretty suspicious. Probably a data-entry error, human error, bug or design error from data pipeline/source, or even a result of badly merging stuff.


### How to find duplicate values properly
SQL columns by default allow duplicate values, so how do we find duplicates and not get mislead by normal data? Us ethe `.duplicated()` method, but pass in arguments to its parameters. This function returns a series of indices that are associated with duplicate rows.
- subset: List of column names to check for duplication.
- keep: Whether to keep first ('first'), last ('last'), or all (False) duplicate values.
```
column_names = ['first_name','last_name','address']
duplicates = df.duplicated(subset=column_names, keep=False)
# Sorting the data so that it's easier to see
df[duplicates].sort_values(by='first_name')
```
So if two or more rows have the same first_name, last_name, and address columns, then they're included in the resulting 'duplicates' data frame. We do 'keep=False' so that we get all rows.
- complete duplicate: Rows in a dataframe that are identical across all columns.
- incomplete duplicate: Rows in a dataframe that are identical based on a subset of columns, rather than all columns.
- 
### Treating duplicate values
1. Use the `df.drop_duplicates()` method. This returns a dataframe with the duplicate rows removed. A by default, rows are considered duplicate when all of their columns match, but you can specify a subset so that if all of the columns in that subset match, then the rows are removed. You can specify the 'keep' argument to keep the first row whilst removing all duplicates, or pass in 'last' to keep the last row. Just don't do 'False' since that just keeps all of the duplicates, and you aren't really achieving anything with that.
2. You can also attempt to combine duplicate rows. If two rows are duplicates, but maybe the 'weight' values are different. You can compute the mean 'weight' value, and put that all in one row. Whether you use mean, min, max, or some other statistical way depends on your data situation.
```
column_names = ['first_name','last_name','address']
summaries = {'height':'max','weight','mean'}
height_weight = df.groupby(by=column_names).agg(summaries).reset_index() 

duplicates = height_weight.duplicated(subset=column_name, keep=False)

# Shouldn't have any duplicate rows show up.
height_weight[duplicates].sort_values(by='first_name')
```

In [ ]:
'''
- Ex. 1: Finding duplicates
1. Find duplicated rows of ride_id. Set keep to 'False' so you actually see all of the duplicate values.
2. Sort the duplicate rows by ride_id.
3. Print the ride_id, duration, and user_birth_year from the duplicate rides rows 
'''
duplicates = ride_sharing.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')
print(duplicated_rides[['ride_id','duration','user_birth_year']])

In [ ]:
'''
- Ex. 2: Treating duplicates

1. Drop complete duplicates from ride_sharing. If the ride_id, user_birth_year, and durations are equal, then drop it. T
2. Create statistics dictionary for aggregation function since we plan to get summary statistics. So we're getting the minimum birth year, and getting the mean for 'duration'.
3. Group the rows by 'ride_id', so we'll create groups, and each group will have rows that have matching ride_id values. Then for the rows in each group, 
we'll calculate summary statistics for the user birth year and duration. We do 'reset_index()' to convert the 'groupby' object back itno a dataframe.
'''
ride_dup = ride_sharing.drop_duplicates(subset=['ride_id','user_birth_year','duration'])
statistics = {'user_birth_year': 'min', 'duration': 'mean'}
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

'''
4. Attempt to find any remaining duplicates based on 'ride_id'. Pass in 'keep=False' to get all duplicates, and then all rows with duplicates 
are marked with true. Remember this is a series of the row indexes where 'True' means duplicate, and false means it's not a duplicate.
5. Filter the the ride_unique to get al lrows where ride_id is duplicated. Storing them in duplicated rides.
'''
# Find duplicated values again
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# Assert duplicates are processed
assert duplicated_rides.shape[0] == 0

# Handling text and categorical data

## Membership constraints
Categorical data has 'discrete' values or values in a finite set of values. For example, 'Marriage Status' can only be 'unmarried' or 'married'. Something like 'Loan Status' could only have values 'default', 'payed', or 'no_loan'. 

There could be issues with categorical data, where the categories are invalid, missing, etc. Let's see how we can treat these 

### How to deal with category 
1. Dropping data
2. Remapping categories
3. Inferring the category value based on other values in the row

### Pandas Joins
Allows you to combine data from two DataFrames based on a common column or index. This operation is similar to SQL joins and is useful for merging datasets that share some relationship. There are main four types of joins:
1. Inner Join: `result = pd.merge(df1,df2, on="some_column", how="inner"`. So it'll return the rows from both df1 and df2, where the 'some_column' values match.
2. Left Join: `result = pd.merge(df1, df2, on='key_column', how='left')`. So it'll return All rows from the left DataFrame (df1), and matching rows from df2. If there were no matching rows from df2, for a particular row in df1, then the rows from df2 will be NaN.
3. Right Join: Same idea as left join, but with the right side.
4. Outer Join: Returns all rows from both DataFrames, and matches where possible. If there's no match, then NaN is used for missing values.

Given two data frames, you can do join operations
- Anti-joins: Return columns that are in dataframe A and not in dataframe B.
- Inner joins: Return columns that are in both A and B.

### Errors we can have

#### Value inconsistency
- Inconsistent fields: 'married', 'Maried', 'UNMARRIED', 'not married '. You can fix some of this by upper or lowercasing everything.
- Trailing or leading whitespace. Fix this by getting rid of the whitespace
In general you're using `.str.strip()`, `.str.lower()`, and `.str.upper()`

#### Creating or remapping categories
- You can create categories
```
# Create category names and ranges  
group_names = ["0-200K", "200K-500K", "500K+"]
ranges = [0, 200000, 500000, np.inf]

demographics['income_group'] = pd.cut(demographics['household_income'], bins=ranges, labels=group_names)
demographics[['income_group', 'household_income']]
```
And the output would look like this:
```
     income_group    household_income
0    0-200K          189243
1    500K+           778533
```
- You can also map categories to fewer ones, in order to reduce the amount of categories in a given column. So here, we replace any entries containing 'Microsoft', 'MacOS', and 'Linux' to 'DesktopOS'. You get the idea. And then we do .unique() to print out all of the unique values of the 'operating_system' series!
```
mapping = {
    "Microsoft": "DesktopOS",
    "MacOS": "DesktopOS",
    "Linux": "DesktopOS",
    "IOS": "MobileOS",
    "Android": "MobileOS"
}
devices['operating_system'] = devices['operating_system'].replace(mapping)
devices['operating_system'].unique()
```


In [3]:
'''
- Ex. 1: The main thing is the idea that we're printing out the unique values in each of the columns. This allows us to see the possible values for these 
columns. Assume we have a 'categories' data frame that contains all the correct possible values of the survey columns.

'''
print("Cleanliness: ", airlines['cleanliness'].unique(), "\n")
print("Safety: ", airlines['safety'].unique(), "\n")
print("Satisfaction: ", airlines['satisfaction'].unique(), "\n")

# Find all categories values in the first set that aren't in the categories['cleanliness'] series; this finds all invalid categories
cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])

# Get all rows with invalid categories; this just returns a series of index and true or false
cat_clean_rows = airlines['cleanliness'].isin(cat_clean)

# Plug that into the dataframe to get all the rows that have an invalid category
print(airlines[cat_clean_rows])

Cleanliness:  ['Clean' 'Average' 'Somewhat clean' 'Somewhat dirty' 'Dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe' 'Somewhat unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satsified' 'Somewhat unsatisfied'
 'Very unsatisfied'] 



In [ ]:
'''
- Ex. 2: Here the main idea is that we make all of the values in 'dest_region' lowercased, and also we remapped a value 'eur'. So 'eur' represents 'europe', but we already have a 'europe' series value, so remap all rows with 'eur' to 'europe'.
'''
# Print unique values of both columns
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

# Lower dest_region column and then replace "eur" with "europe"
airlines['dest_region'] = airlines['dest_region'].str.lower() 
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

# Remove white spaces from `dest_size`
airlines['dest_size'] = airlines['dest_size'].str.strip()

# Verify changes have been effected
print(airlines['dest_region'])
print(airlines['dest_size'])

In [ ]:
'''
- Ex. 3: 

- Rules: 
1. wait_type: 'short' for 0-60 min, 'medium' for 60-180 and long for 180+. We have an existing 'wait_mins' column you can use.
2. day_week: 'weekday' if day is in the weekday, 'weekend' if day is in the weekend. We have an existing 'day' column that you can derive from.

demographics['income_group'] = pd.cut(demographics['household_income'], bins=ranges, labels=group_names)
'''

# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column based on the 'wait_min'
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins=label_ranges, labels=label_names)


# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}
airlines['day_week'] = airlines['day'].replace(mappings)

## Cleaning text data

Text data consists of things such as names, phone numbers, emails, passwords, and any other data that's going to be represented as strings.

### Common text data problems
- Inconsistency: Is the phone number represented as `+96171679912` or `0096171679912`. We need to have a uniform way of processing it.
- Constraint violatiosn: Passwords need to be at least 8 characters, and meet other conditions.
- Typos: A phone number with a typo like `+961.471.679912` needs to be fixed.

#### Phone number example
Let's say that we want all of our phone numbers in the form `00xxxxxxxxxxx`. Let's say some of our data is in form `+x-xxx-xxx-xxxx`. Let's convert things. Let's also handle the case where incomplete phone numbers like `xx-xxxx` or something similar is converted to 
```
phones['Phone number'] = phones['Phone number'].str.replace("+", "00")
phones['Phone number'] = phones['Phone number'].str.replace("-", "")

# Returns a series of the lengths of each phone number
digitLength = phones['Phone number'].str.len()

# digitLength < 10 creates a boolean series.
# Then phones.loc selects all rows in the 'Phone number' column where the condition is true, derived from the series value.
phones.loc[digitLength < 10, "Phone number"] = np.nan 
```


In [ ]:
'''
- Ex. 1: Do some string manipulate to remove honorifics from the 'full_name' column.
1. Replace "Dr." with empty string ""
2. Replace "Mr." with empty string ""
3. Replace "Miss" with empty string ""
4. Replace "Ms." with empty string ""
'''

airlines['full_name'] = airlines['full_name'].str.replace("Dr.","")
airlines['full_name'] = airlines["full_name"].str.replace("Mr.", "")
airlines["full_name"] = airlines['full_name'].str.replace("Miss", "")
airlines['full_name'] = airlines["full_name"].str.replace("Ms.", "")

# Assert that full_name has no honorifics
assert airlines['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

In [ ]:
'''
- Ex. 2: Do some string manipulate to remove honorifics from the 'full_name' column.
1. Store length of each row in survey_response column
2. Find rows in airlines where resp_length > 40
3. Assert minimum survey_response length > 40
4. Print values of the the new 'survey_response' 
'''
resp_length = airlines['survey_response'].str.len()
airlines_survey = airlines[resp_length > 40]
assert airlines_survey['survey_response'].str.len().min() > 40
print(airlines_survey['survey_response'])

# Advanced data problems

Addressing issues such as ensuring all weights are in kilograms instead of pounds. Then we'll go into more practice and detail to ensure that missing values don't negatively impact our analysis. 


## Unit Uniformity
We need to unit uniformity. Data such as temperature needs to all be in Celsius or Fahrenheit, pick one! If we have weights in kilograms, we should know how to be able to programmatically convert those values into pounds or other forms as well. The same goes for dates and also currency.


#### Examples
Here's an example based on temperatures. We can assume that the `temperatures` DataFrame has values in fahrenheit by default
```
# We can assume that values above 40 represent fahrenheit
temp_fahrenheit = temperatures.loc[temperatures['Temperature'] > 40, 'Temperature']

# Apply a function on the series?
temp_celsius = (temp_fahrenheit - 32) * (5/9)

# Assert that the conversion is correct
temperatures.loc[temperatures['Temperature'] > 40, 'Temperature'] = temp_celsius
assert temperature ['Temperature'].max() < 40
```
Here's an example based on dates. Assume that we have 3 formats for dates `27/27/19`, `03-29-19`, or `March 3rd, 2019`. 
```


birthdays['Birthday'] = pd.to_datetime(birthdays['Birthday'], infer_datetime_format=True, )

```
### Treating ambiguous date data
Is `2019-03-08` in August or March. Here are a couple of ways you can handle this:
1. Convert to Na or missing and treat accordingly.
2. Infer the format by understanding the data source.
3. Infer the format by understanding previous and subsequent data in DataFrame.
4. 

In [ ]:
'''
+ Ex. 1: We're working with banking data. So let's do a simple conversion example

- Data columns:
1. acct_amount: Amount of money stored in the accounts
2. acct_cur: Currency type
3. inv_amount: Amount invested
4. account_opened: Account opening date. When the account was made.
5. last_transaction: Date of the most recent transaction
''''

# Find values of acct_cur that are equal to 'euro'; our filter
acct_eu = banking['acct_cur'] == "euro"

# conversion rate from euro to usd 
conversion_rate = 1.1

# For every row that has a 'euro' currency, update its 'acct_amount' to be in USD
banking.loc[acct_eu, "acct_amount"] = banking.loc[acct_eu, "acct_amount"] * conversion_rate 

# For every row with the euro currency, convert the currency type to dollar
banking.loc[acct_eu, "acct_cur"] = banking.loc[acct_eu, "acct_cur"] = "dollar"

# Assert that only dollar currency remains
assert banking['acct_cur'].unique() == ['dollar']

In [ ]:
'''
+ Ex. 2: Handling the converting of date formats. So let's uniformalize the date formatting.
So when you first try pd.to_datetime() on the 'account_opened' series, you get this error
'ValueError: month must be in 1..12'. Alongside seeing the output, it just means that our dates aren't in a uniform format, which causes the error when we try to change the strings into date objects.


Before output: 
    0          2018-03-05
    1            21-01-18
    2    January 26, 2018
    3            21-14-17
    4            05-06-17
    
- How does 'infer_datetime_format' work?
Instructs Pandas to automatically guess the date format of each string in the series, as this is built in to handle a variety of formats such as:
1. YYYY-MM-DD (e.g., "2018-03-05")
2. DD-MM-YY or MM-DD-YY (e.g., "21-01-18" or "05-06-17")
3. Month DD, YYYY (e.g., "January 26, 2018")

Also Pandas is generally good at parsing a widely variety of date formats, including those with different delimiters such as '/', '-', or spaces. Dates are converted into 'yyyy-mm-dd' format.

After output:
0   2018-03-05
1   2018-01-21
2   2018-01-26
3          NaT, '14' isn't a valid month
4   2017-05-06
Name: account_opened, dtype: datetime64[ns]
'''

# Print the header of account_opened
print(banking['account_opened'].head())

# Convert account_opened to datetime
banking['account_opened'] = pd.to_datetime(
    banking['account_opened'],
    infer_datetime_format = True,
    errors = 'coerce'
)

# Get year of account opened
banking['acct_year'] = banking['account_opened'].dt.strftime("%Y")

# Print acct_year
print(banking['acct_year'])

## Cross field validation
The use of fields to check other fields, or you can think of it as using multiple fields to ensure your data is valid. For example using the 'first_class', 'business_class', and 'economy_class' and summing those up to make sure that they match the 'total_passengers' field!
```
sum_classes = flights['economy_class','business_class','first_class'].sum(axis=1)

passengers_equals = sum_classes == flights['total_passengers']

inconsistent_passengers_rows = flights[~passengers_equals]
consistent_passengers_rows = flights[passengers_equals]
```
Another example could be comparing a user's 'age' vs their 'Birthday' field.
```
import pandas as pd
import datetime as dt

# Convert all string values to date time objects
users['Birthday'] = pd.to_datetime(users['Birthday'])
today = dt.date.today();

# Manually calculate the age for every value in the series. Then create a filter
calculated_age = today.year - users['Birthday'].dt.year
age_equals = calculated_age == users['Age']

# Find and filter out rows with inconsistent and consistent ages
inconsistent_age_rows = users[~age_equals]
consistent_age_rows = users[age_equals]
```

### How to treat inconsistencies

1. Dropping data
2. Set to missing and maybe impute it (derive it from other fields).
3. Apply rules or what you know about your dataset that's appropriate for your specific situation.


In [ ]:
'''
+ Ex. 1: Maintaining data integrity. Ensuring the funds all sum up to the investment amount associated with the account. Ensuring also the ages were good.
'''
# Store fund columns to sum against
fund_columns = ['fund_A', 'fund_B', 'fund_C', 'fund_D']
sum_funds = banking[fund_columns].sum(axis=1)

# Find rows where fund_columns row sum == inv_amount
inv_equ = sum_funds == banking['inv_amount']

# Store consistent and inconsistent data
consistent_inv = banking[~inv_equ]
inconsistent_inv = banking[inv_equ]

# Convert all string values to date time objects
banking['age'] = pd.to_datetime(banking['birth_date'])
today = dt.date.today();
ages_manual = today.year - banking['birth_date'].dt.year
age_equals = ages_manual == banking['Age']

inconsistent_ages = banking[~age_equals]
consistent_ages = banking[age_equals]

## Missing Data
When no data value is stored for a value in a row. Missing data is typically represented by 'NA', 'nan', 0, or '.' This commonly happens due to a technical or human error.

### Correlations between missing data
1. Missing Completely at Random (MCAR)
- Definition: Data is missing completely at random when the likelihood of a data point being missing is unrelated to any other data, including both observed and unobserved data.
- Example: Imagine you're collecting survey data, and some respondents accidentally skipped a question due to a random software glitch. The missing data doesn't depend on any specific trait of the respondent or other answers they've given; it's purely random.
- Implication: Since there's no pattern to the missing data, it doesn't bias the analysis. However, you may lose some statistical power because of the reduced sample size.
2. Missing at Random (MAR)
- Definition: Data is missing at random when the missingness is related to other observed data but not to the value of the data that is missing itself.
- Example: Suppose you're analyzing a dataset of students' grades, and you notice that older students are more likely to have missing grades because they tend to skip more classes. The missing grades are related to the age of the students (which is observed), but not to the actual grades themselves.
- Implication: The missing data can introduce bias if not properly accounted for. However, because the missingness is related to other observed variables, techniques like imputation (filling in missing data based on observed data) can be used effectively.
3. Missing Not at Random (MNAR)
- Definition: Data is missing not at random when the missingness is related to the value of the data itself that is missing. In other words, there's a systematic relationship between the propensity of missingness and the unobserved data.
- Example: Consider a survey question about income where high-income individuals are more likely to leave the question unanswered because they feel it's too personal. The likelihood of missing data is directly related to the income level, which is the value of the missing data itself.

In [ ]:
'''
+ Ex. 1: 

'''
# Print number of missing values in banking
print(banking.isna().sum())

# Visualize missingness matrix
msno.matrix(banking)
plt.show()

# Isolate missing and non missing values of inv_amount
missing_investors = banking[banking['inv_amount'].isna()]
investors = banking[~banking['inv_amount'].isna()]

# Sort banking by age and visualize
banking_sorted = banking.sort_values(by = 'age')
msno.matrix(banking_sorted)
plt.show()